In [1]:
from llm_response import MentalHealthBot

query = "I am feeling very anxious and overwhelmed. What should I do?"

bot = MentalHealthBot()
print(bot.chat_stream(query))

/Users/satyamgoyal/anaconda3/envs/hermes/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/satyamgoyal/anaconda3/envs/hermes/lib/python3.9/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Starting Bot -----------------------------------###
Initializing RAG system
Initializing LLM
Creating RAG chain
Creating new chat history for session_id abc123


No relevant docs were retrieved using the relevance score threshold 0.6


Human: I am feeling very anxious and overwhelmed. What should I do?
AI: Okay, I understand you're feeling anxious and overwhelmed. That's tough, and it's important to address it. Here's a breakdown of things you can try, ranging from immediate relief to longer-term strategies:

**I. Immediate Relief (Right Now):**

*   **Breathe:** This is the most crucial first step.  Anxiety often leads to shallow breathing, which worsens the feeling. Try one of these techniques:
    *   **4-7-8 Breathing:** Inhale deeply through your nose for a count of 4, hold your breath for a count of 7, and exhale slowly through your mouth for a count of 8. Repeat this 4-5 times.
    *   **Box Breathing:** Inhale for 4, hold for 4, exhale for 4, hold for 4. Repeat several times.
    *   **Diaphragmatic Breathing:** Place one hand on your chest and the other on your stomach. Breathe in deeply, focusing on expanding your stomach while keeping your chest relatively still. Exhale slowly.

*   **Grounding Techniques 